In [ ]:
#!pip install --upgrade azureml-sdk azureml-dataprep matplotlib

In [ ]:
#Get AzureML Workspace config
#Setup environment using samples here: https://github.com/Azure/MachineLearningNotebooks/tree/master/setup-environment
from azureml.core import Workspace, Dataset
workspace = Workspace.from_config()
#print(workspace)

In [ ]:
#Avro files location pre-registered as a files dataset. Ensure the dataset path is of the format folder/** to pick up all partitions and folders inside.
dataset_name="eventhub-capture-avro"
dataset = Dataset.get_by_name(workspace, name=dataset_name)
print(dataset.name)


In [ ]:
# mount dataset manually instead of using
# with dataset.mount() as mount_context:
import os
mount_context=dataset.mount()
mount_context.start()
# See where we mounted the file dataset
print(mount_context.mount_point)
# List files in there
print(os.listdir(mount_context.mount_point))

In [ ]:
#Display Recursive - folder and files 
dataset_mount_folder = mount_context.mount_point
for root,d_names,f_names in os.walk(dataset_mount_folder):
	print(root, d_names, f_names)

In [ ]:
#pickup a sample File
files = os.listdir(os.path.join(dataset_mount_folder,'eventhub1/0/2021/09/08/06/35'))
print(files)

In [ ]:
# !pip install fastavro
from fastavro import writer, reader, parse_schema

with open(os.path.join(dataset_mount_folder,'eventhub1/0/2021/09/08/06/35/14.avro'), 'rb') as fo:
    avro_reader = reader(fo)
    records = [r for r in avro_reader]
print(records)

In [ ]:
#Add records to a pandas DataFrame
import pandas as pd
df = pd.DataFrame.from_records(records)
df.head(2)

In [ ]:
# Unmount dataset
mount_context.stop()